KeyboardInterrupt: 

In [8]:
!pip install fastapi nest-asyncio uvicorn python-multipart tensorflow pillow -q

from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import uvicorn
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
import io
import logging
from google.colab.output import eval_js
import threading

# Настройка логгирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Загрузка модели
logger.info("Загружаем модель...")
model = MobileNetV2(weights='imagenet')
logger.info("Модель готова к работе!")

# Создание FastAPI приложения
app = FastAPI(title="Object Recognition API")

# Настройка CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

async def process_image(file_bytes):
    try:
        img = Image.open(io.BytesIO(file_bytes))
        if img.format not in ['JPEG', 'PNG', 'JPG']:
            raise HTTPException(status_code=400, detail="Поддерживаются только JPEG/PNG")
        if len(file_bytes) > 5 * 1024 * 1024:
            raise HTTPException(status_code=400, detail="Слишком большой файл (макс. 5MB)")

        img = img.resize((224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        predictions = model.predict(img_array)
        results = decode_predictions(predictions, top=3)[0]

        return [
            {"object": result[1], "confidence": float(result[2])}
            for result in results
        ]
    except Exception as e:
        logger.error(f"Ошибка обработки: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()
    results = await process_image(contents)
    return {"predictions": results, "top_prediction": results[0]}

@app.get("/")
async def test_interface():
    return """
    <h1>Сервис распознавания объектов</h1>
    <form action="/predict" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*">
        <button type="submit">Распознать</button>
    </form>
    """

def start_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

if __name__ == "__main__":
    nest_asyncio.apply()
    threading.Thread(target=start_server, daemon=True).start()
    public_url = eval_js("google.colab.kernel.proxyPort(8000)")
    print(f"API доступен по адресу: {public_url}")

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-10' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

API доступен по адресу: https://8000-m-s-3bs3on7d34kd-a.asia-east1-1.prod.colab.dev


In [24]:
# Установка зависимостей
!pip install fastapi uvicorn python-multipart tensorflow pillow nest-asyncio pydantic -q

from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import List, Optional
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
import io
import logging
import nest_asyncio
from google.colab.output import eval_js
import threading

# Настройка логгирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Инициализация модели
logger.info("Инициализация модели MobileNetV2...")
model = MobileNetV2(weights='imagenet')
logger.info("Модель успешно загружена")

# Создание FastAPI приложения
app = FastAPI(title="Object Recognition API")

# Включение CORS
app.add_middleware(
    "fastapi.middleware.CORSMiddleware",
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Модели Pydantic для ответа API
class PredictionResult(BaseModel):
    object: str
    confidence: float

class APIResponse(BaseModel):
    success: bool
    predictions: List[PredictionResult]
    top_prediction: PredictionResult
    error: Optional[str] = None

def process_image_file(file: UploadFile) -> np.ndarray:
    """Обработка загруженного файла изображения"""
    try:
        contents = file.file.read()
        if len(contents) > 5 * 1024 * 1024:
            raise HTTPException(status_code=400, detail="Файл слишком большой (макс. 5MB)")

        img = Image.open(io.BytesIO(contents))
        if img.format not in ['JPEG', 'PNG', 'JPG']:
            raise HTTPException(status_code=400, detail="Неподдерживаемый формат изображения")

        img = img.resize((224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        return preprocess_input(img_array)

    except Exception as e:
        logger.error(f"Ошибка обработки файла: {str(e)}")
        raise HTTPException(status_code=400, detail=f"Ошибка обработки изображения: {str(e)}")

@app.post("/predict", response_model=APIResponse)
async def predict_object(file: UploadFile = File(...)):
    try:
        processed_image = process_image_file(file)
        predictions = model.predict(processed_image)
        decoded_predictions = decode_predictions(predictions, top=3)[0]

        results = [
            PredictionResult(object=label, confidence=float(conf))
            for (_, label, conf) in decoded_predictions
        ]

        return APIResponse(
            success=True,
            predictions=results,
            top_prediction=results[0]
        )

    except HTTPException as he:
        return JSONResponse(
            content={
                "success": False,
                "error": he.detail,
                "predictions": [],
                "top_prediction": None
            },
            status_code=he.status_code
        )
    except Exception as e:
        logger.error(f"Неожиданная ошибка: {str(e)}")
        return JSONResponse(
            content={
                "success": False,
                "error": "Внутренняя ошибка сервера",
                "predictions": [],
                "top_prediction": None
            },
            status_code=500
        )

@app.get("/")
async def test_interface():
    return """
    <html>
    <body>
        <h1>Сервис распознавания объектов</h1>
        <form action="/predict" method="post" enctype="multipart/form-data">
            <input type="file" name="file" accept="image/*" required>
            <button type="submit">Распознать объекты</button>
        </form>
        <div id="result" style="margin-top:20px;"></div>
        <script>
            document.querySelector('form').addEventListener('submit', async (e) => {
                e.preventDefault();
                const formData = new FormData(e.target);
                const response = await fetch('/predict', {
                    method: 'POST',
                    body: formData
                });
                const result = await response.json();
                document.getElementById('result').innerHTML = `
                    <h3>Результаты:</h3>
                    <p>Наиболее вероятный объект: <strong>${result.top_prediction.object}</strong></p>
                    <p>Уверенность: <strong>${(result.top_prediction.confidence * 100).toFixed(2)}%</strong></p>
                    <h4>Все варианты:</h4>
                    <ul>
                        ${result.predictions.map(p => `<li>${p.object} (${(p.confidence * 100).toFixed(2)}%)</li>`).join('')}
                    </ul>
                `;
            });
        </script>
    </body>
    </html>
    """

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

if __name__ == "__main__":
    nest_asyncio.apply()
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()

    public_url = eval_js("google.colab.kernel.proxyPort(8000)")
    print(f"Сервис доступен по адресу: {public_url}")
    print("Отправляйте POST-запросы с изображениями на /predict")

INFO:     Started server process [700]
INFO:     Waiting for application startup.
INFO:     ASGI 'lifespan' protocol appears unsupported.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use


Сервис доступен по адресу: https://8000-m-s-3bs3on7d34kd-a.asia-east1-1.prod.colab.dev
Отправляйте POST-запросы с изображениями на /predict


In [25]:
{
  "success": True,
  "predictions": [
    {"object": "labrador", "confidence": 0.95},
    {"object": "retriever", "confidence": 0.03}
  ],
  "top_prediction": {"object": "labrador", "confidence": 0.95}
}

{'success': True,
 'predictions': [{'object': 'labrador', 'confidence': 0.95},
  {'object': 'retriever', 'confidence': 0.03}],
 'top_prediction': {'object': 'labrador', 'confidence': 0.95}}

In [33]:
# 1. Установка Docker в Colab
!curl -fsSL https://get.docker.com -o get-docker.sh
!sh get-docker.sh

# 2. Создаем файл requirements.txt правильно
with open('requirements.txt', 'w') as f:
    f.write("""fastapi==0.68.1
uvicorn==0.15.0
python-multipart==0.0.5
tensorflow==2.6.0
pillow==8.3.2
""")

# 3. Создаем основной файл приложения
with open('app.py', 'w') as f:
    f.write("""from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
import io
import uvicorn

app = FastAPI()
model = MobileNetV2(weights='imagenet')

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        img = Image.open(io.BytesIO(await file.read()))
        img = img.resize((224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        preds = model.predict(img_array)
        results = decode_predictions(preds, top=3)[0]

        return JSONResponse(content={
            "success": True,
            "predictions": [{"label": label, "confidence": float(conf)}
                          for (_, label, conf) in results]
        })
    except Exception as e:
        return JSONResponse(
            content={"success": False, "error": str(e)},
            status_code=400
        )

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
""")

# 4. Создаем Dockerfile
with open('Dockerfile', 'w') as f:
    f.write("""FROM python:3.9-slim
WORKDIR /app
COPY . .
RUN apt-get update && \\
    apt-get install -y --no-install-recommends gcc python3-dev && \\
    pip install --no-cache-dir -r requirements.txt && \\
    apt-get remove -y gcc python3-dev && \\
    apt-get autoremove -y && \\
    rm -rf /var/lib/apt/lists/*
EXPOSE 8000
CMD ["python", "app.py"]
""")

# 5. Собираем и запускаем контейнер
!docker build -t colab-app .
!docker run -d -p 8000:8000 --name my-app colab-app

# 6. Проверяем работу
import time
time.sleep(5)
print("Сервис доступен по адресу: http://localhost:8000")
print("Пример тестового запроса:")
print("""
import requests
url = "http://localhost:8000/predict"
files = {'file': open('test.jpg', 'rb')}
response = requests.post(url, files=files)
print(response.json())
""")

# Executing docker install script, commit: 7040dd2bf115a359317b1de84de611aeabcb7bc2
+ sh -c apt-get -qq update >/dev/null
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu jammy stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
+ sh -c DEBIAN_